# import

In [ ]:
#export
from fastai.vision import *

In [ ]:
#export
import cv2

In [ ]:
#export
import os

# functions

In [ ]:
#export
def get_y(x, ds_rootdir, imgdir, maskdir):
    yfn = os.path.join(ds_rootdir, maskdir, '%s_mask%s' % (x.stem, x.suffix))
    #print(yfn, x.stem, x.suffix)
    return yfn

In [66]:
#export
def get_databunch(ds_root_dir = 'dataset_20200708', ds_imgdir = 'image'
                  , ds_maskdir = 'mask', bs = 16, valid_pct = 0.2
                  , device = torch.device('cpu')
                  , transforms = get_transforms(max_zoom = 1.)):
    '''
    获取databunch
    参数：
        ds_root_dir：数据集的根目录
        ds_imgdir：图片子目录
        ds_maskdir: mask图片子目录
        bs：batch_size
        valid_pct:验证集百分比
        device: 设备
        transforms: 无缩放，其余默认参数。
    返回值：
        databunch
    '''
    data = SegmentationItemList.from_folder( \
                    os.path.join(ds_root_dir, ds_imgdir))
    
    data = data.split_by_rand_pct(valid_pct)
    data = data.label_from_func( \
            partial(get_y, ds_rootdir = ds_root_dir, imgdir = ds_imgdir, maskdir = ds_maskdir) \
            , classes=['0', '255'])
    
    #import pdb; pdb.set_trace()
    if transforms is not None:
        data = data.transform(transforms, tfm_y = True)
    data = data.databunch(bs=bs, num_workers = 0, device = device)
    data = data.normalize(imagenet_stats)
    
    '''
    data = (UnetSegmentationItemList
        .from_folder(os.path.join(ds_root_dir, ds_imgdir))
        .split_by_rand_pct(0.2)
        .label_from_func(partial(get_y, ds_rootdir = ds_root_dir, imgdir = ds_imgdir, maskdir = ds_maskdir), classes=['0', '255'])
        .transform(get_transforms(), tfm_y=True)
        .databunch(bs=bs, num_workers = 0)
        .normalize(imagenet_stats)
       )
    '''    
    return data

#data = get_databunch(bs = 4)
#data.show_batch()

# test

In [68]:
device = torch.device('cuda')

In [70]:
data = get_databunch(bs = 4, device = device)

In [ ]:
#用自定义的transform调用
# data = get_databunch(bs = 4
#     , transforms = get_transforms(max_zoom = 2.
#                                   , max_rotate = 60.
#                                   , max_lighting = 0.6
#                                   , max_warp = 0.5))

In [71]:
data = get_databunch(bs = 4, transforms = None)

In [ ]:
data

In [ ]:
data.show_batch()

In [ ]:
data.show_batch(ds_type=DatasetType.Valid)

In [14]:
idx = 0

In [73]:
empt = 0
for i in range(len(data.valid_ds)):
    na = data.train_ds[i][1].data.numpy()
    na[na < 128] = 0
    na[na >= 128] = 1
    #print(na.sum(), na.sum() / 512 / 512)
    #data.train_ds[idx][1]
    if na.sum() / 512 / 512 < 0.04:
        empt += 1
print(empt / len(data.valid_ds))        

0.625


# export

In [ ]:
!python notebook2script.py --fname 'databunch.ipynb' --outputDir './exp/'